In [3]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

# ConversationSummaryBufferMemory : ConversationSummaryMemory + ConversationBufferMemory
# saving how many memories we sending to
# when hit the limit, it will summarize old messages not remove it
# llm -> When you use this memory - this causing cost
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=150,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Dean, I live in Singapore", "Wow that's so awsome")

In [4]:
get_history()

{'history': [HumanMessage(content="Hi I'm Dean, I live in Singapore"),
  AIMessage(content="Wow that's so awsome")]}

In [5]:
add_message("Singapore is so hot", "I wish I could go!")

In [6]:
get_history()

{'history': [HumanMessage(content="Hi I'm Dean, I live in Singapore"),
  AIMessage(content="Wow that's so awsome"),
  HumanMessage(content='Singapore is so hot'),
  AIMessage(content='I wish I could go!')]}

In [8]:
add_message("How far Singapore from Argentina?", "I don't know! that's super far!")

In [9]:
get_history()

{'history': [HumanMessage(content="Hi I'm Dean, I live in Singapore"),
  AIMessage(content="Wow that's so awsome"),
  HumanMessage(content='Singapore is so hot'),
  AIMessage(content='I wish I could go!'),
  HumanMessage(content='How far Singapore from Argentina?'),
  AIMessage(content="I don't know! that's super far!"),
  HumanMessage(content='How far Singapore from Argentina?'),
  AIMessage(content="I don't know! that's super far!")]}

In [ ]:
add_message("How far is Brazil from Argentina?", "I don't know! that's super far!")